In [22]:
%reset -f

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import numpy as np
import json
import os
import urllib

from imageai.Classification import ImageClassification

from PIL import Image
import requests

pd.set_option('display.max_columns', None)

from tqdm import tqdm

tqdm.pandas()

from datetime import datetime

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Horego-GCP-Keys.json')
project_id = 'horego-big-query'

# Make clients.
horego_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import psycopg2.pool
def db_connect_cashbac_nopool(host,database,user,password):
    """
    :return:
    """
    connection = psycopg2.connect(host=host,
                              database=database,
                              user=user,
                              password=password,
                              port=5432)
    return connection

# IMPORT BQ NEW WAY

In [3]:
import MyFunc
# from MyFunc import cleaning_data_merchants,filtering_data_merchants,get_post_code,upload_file_to_GCS,create_table_from_gcs,is_on_radius,get_hot_area
from MyFunc import *

from importlib import reload
reload(MyFunc)
from MyFunc import *
# cleaning_data_merchants,filtering_data_merchants,get_post_code,upload_file_to_GCS,create_table_from_gcs,is_on_radius,get_hot_area

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
%%time

query_string=f""" 

SELECT * 
FROM `cashbac-31433.cashbac_datalake_prod.outcrapper_google_photos` table1
limit 1

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

1
CPU times: user 53.8 ms, sys: 26.9 ms, total: 80.6 ms
Wall time: 1.14 s


In [5]:
%%time
import os
from google.cloud import bigquery

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

scrappe_date='20231114'
# task_id = '202310261126558bb8'

path = 'outcrapper_raw/photos/'
files = os.listdir(path)
files = [f for f in files if os.path.isfile(path+'/'+f)]
files.sort()

for file in files:
    if '.json' in file and int(file[:8])>=int(scrappe_date):
    # if '.json' in file and task_id in file:
        print(file)
        f = open(f'{path}{file}')
        data = json.load(f)
        data = pd.json_normalize(data)
        
        data = data[(data.photo_url.isnull()==False)]
        data = data[~data.photo_url_big.str.contains('streetview')]
        # data = data[~data.photo_id.isin(existing_list.photo_id)]
        
        date_str = file[:8]
        date_format = '%Y%m%d'
        
        data['scrappe_date'] = datetime.strptime(date_str, date_format).strftime('%Y-%m-%d')
        # data=cleaning_photos(data)

        my_schema=[]
        save_columns=[]
        for column in df_temp.columns :
            if column in data.columns :
                save_columns.append(column)
                my_schema.append(bigquery.SchemaField(column, "STRING"))
        
        # print(len(data[save_columns]))
        data[save_columns].to_csv('results/temp_upload_photos.csv',sep=';',index=False)
        
        # my_schema=[]
        # for column in data.columns :
        #     my_schema.append(bigquery.SchemaField(column, "STRING"))
            
        #############################################################################################
        #BQ DATALAKE HOREGO
        src_path_file='results/temp_upload_photos.csv'
        target_path_file=f"scrapping_results/photos/{file.replace('.json','.csv')}"
        bucket_name='horego-bq'
        upload_file_to_GCS(horego_key,bucket_name,src_path_file,target_path_file)

        table_schema = "horego_datalake_dev"
        table_name = "outcrapper_google_photos"
        gcs_path_uri = f"gs://{bucket_name}/scrapping_results/photos/{file.replace('.json','.csv')}"
        create_table_from_gcs(horego_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        
        #############################################################################################
        #############################################################################################
        #BQ DATALAKE CASHBAC
        bucket_name='cashbac_datalake'
        src_path_file='results/temp_upload_photos.csv'
        target_path_file=f"scrapping_results/photos/{file.replace('.json','.csv')}"
        upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

        table_schema = "cashbac_datalake_prod"
        table_name = "outcrapper_google_photos"
        gcs_path_uri = f"gs://{bucket_name}/scrapping_results/photos/{file.replace('.json','.csv')}"
        create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')
        #############################################################################################
        

20231114022617792f_photos_places_with_no_photos_jakut_jakbar_jakpus.json
Uploading results/temp_upload_photos.csv
Path in GCS: horego-bq/scrapping_results/photos/20231114022617792f_photos_places_with_no_photos_jakut_jakbar_jakpus.csv
table on : horego_datalake_dev.outcrapper_google_photos
Uploading results/temp_upload_photos.csv
Path in GCS: cashbac_datalake/scrapping_results/photos/20231114022617792f_photos_places_with_no_photos_jakut_jakbar_jakpus.csv
table on : cashbac_datalake_prod.outcrapper_google_photos
20231114022842e088_photos_places_with_no_photo_menus_jaktim_-_jaksel.json
Uploading results/temp_upload_photos.csv
Path in GCS: horego-bq/scrapping_results/photos/20231114022842e088_photos_places_with_no_photo_menus_jaktim_-_jaksel.csv
table on : horego_datalake_dev.outcrapper_google_photos
Uploading results/temp_upload_photos.csv
Path in GCS: cashbac_datalake/scrapping_results/photos/20231114022842e088_photos_places_with_no_photo_menus_jaktim_-_jaksel.csv
table on : cashbac_data

### IMPORT TO list_insert_to_prod 

In [12]:
query=f""" 

SELECT distinct google_id,place_id
FROM `cashbac-31433.cashbac_datalake_prod.vw_outcrapper_google_photos` 
where scrappe_date >= '{datetime.strptime(scrappe_date, date_format).strftime('%Y-%m-%d')}'

"""

query_result = list(cashbac_bqclient.query(query).result(timeout=None))
print(len(query_result))
if len(query_result)>0:
    data = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

24477


In [13]:
%%time

query_string=f""" 

SELECT column_name
FROM cashbac-31433.cashbac_datalake_prod.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'list_insert_to_prod'

"""

query_result = list(cashbac_bqclient.query(query_string).result(timeout=None))
print(len(query_result))
df_temp = pd.DataFrame(data=[list(x.values()) for x in query_result], columns=list(query_result[0].keys()))

my_schema=[]
save_columns=[]
for column in df_temp.column_name :
    if column in data.columns :
        save_columns.append(column)
        my_schema.append(bigquery.SchemaField(column, "STRING"))

data[save_columns].drop_duplicates().to_csv('results/input_list_insert_to_prod.csv',sep=';',index=False)

5
CPU times: user 72.9 ms, sys: 33.8 ms, total: 107 ms
Wall time: 826 ms


In [14]:
cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

#############################################################################################

# df_temp=pd.read_csv('results/choosen_photos.csv',sep=';', nrows=1)
# my_schema=[]
# for column in df_temp.columns :
#     my_schema.append(bigquery.SchemaField(column, "STRING"))

src_path_file='results/input_list_insert_to_prod.csv'
target_path_file='scrapping_results/input_list_insert_to_prod.csv'
bucket_name='cashbac_datalake'
upload_file_to_GCS(cashbac_key,bucket_name,src_path_file,target_path_file)

table_schema = "cashbac_datalake_prod"
table_name = "list_insert_to_prod"
gcs_path_uri = f"gs://{bucket_name}/scrapping_results/input_list_insert_to_prod.csv"
create_table_from_gcs(cashbac_key,gcs_path_uri,table_schema,table_name,my_schema,'append')

Uploading results/input_list_insert_to_prod.csv
Path in GCS: cashbac_datalake/scrapping_results/input_list_insert_to_prod.csv
table on : cashbac_datalake_prod.list_insert_to_prod
